In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
import pandas as pd
import glob

In [ ]:
autoencoder_model = tf.keras.saving.load_model('autoencoder_model')
classifier = tf.keras.saving.load_model('fake_classifier')
# Create a new model that only includes the encoder part
encoder_model = tf.keras.Model(inputs=autoencoder_model.input, outputs=autoencoder_model.get_layer('dense_14').output)

In [ ]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from pydub import AudioSegment

def audio_to_mel_spectrogram(audio_file_path, max_length=4*22500, sr=22500, n_mels=256):
    #audio_file_path = audio_file_path if type(audio_file_path) == str else audio_file_path.decode('ASCII')
    #audio_file_path = str(audio_file_path).replace('\\\\', os.sep)
    #audio_file_path = audio_file_path.replace('F:', '/mnt/f').replace('//', '/')
    try:
        with open(audio_file_path, 'rb') as fh:
            y, sr = librosa.load(fh, sr=sr)
    except:
        y = AudioSegment.from_file(audio_file_path)
    
    if len(y) > max_length:
        # If the audio is longer than the maximum length, cut it
        y = y[:max_length]
    elif len(y) < max_length:
        # If the audio is shorter, pad it with zeros
        pad_length = max_length - len(y)
        y = np.pad(y, (0, pad_length), mode='constant')
    
    # Calculate the Mel spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_mels)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
    
    return mel_spectrogram

# Generator function for lazy loading of audio data
def audio_data_generator(audio_files):
    for audio_file in audio_files:
        sample = (audio_to_mel_spectrogram(audio_file) + 80)/80
        yield sample.reshape((1, 128, 176, 1))

In [ ]:
# Example usage
spectrogram = next(audio_data_generator(["converted.wav"]))
print(spectrogram.shape)
embedding = encoder_model.predict(spectrogram)
y = classifier.predict(embedding)
y

(1, 128, 176, 1)
1/1 [==============================] - 0s 24ms/step


c:\Users\gabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


array([[0.3797141]], dtype=float32)

In [ ]:
spectrogram = next(audio_data_generator(["reconstructed_audio.wav"]))
print(spectrogram.shape)
embedding = encoder_model.predict(spectrogram)
y = classifier.predict(embedding)
y

(1, 128, 176, 1)
1/1 [==============================] - 0s 49ms/step


c:\Users\gabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


1/1 [==============================] - 0s 116ms/step


array([[0.5655227]], dtype=float32)

In [ ]:
spectrogram = next(audio_data_generator(["recording(42).wav"]))
print(spectrogram.shape)
embedding = encoder_model.predict(spectrogram)
y = classifier.predict(embedding)
y

FileNotFoundError: [Errno 2] No such file or directory: 'Recording\\ (42).m4a'

In [ ]:
spectrogram = next(audio_data_generator(["recording.wav"]))
print(spectrogram.shape)
embedding = encoder_model.predict(spectrogram)
y = classifier.predict(embedding)
y